# 載入套件、設定function、載入模型

In [14]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from dy_resnet import *
import torch.nn.functional as F
import random, time, torchprofile
import torchvision.models as models
from tqdm import tqdm
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# Define a custom dataset class
class ImageNetDataset(Dataset):
    def __init__(self, data_file, root_dir, transform=None):
        """
        Args:
            data_file (string): Path to the file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        # Read the data file
        with open(data_file, 'r') as file:
            self.data = file.readlines()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Process each line
        line = self.data[idx].strip()
        img_path, label = line.split()
        # Complete image path
        img_path = os.path.join(self.root_dir, img_path)
        image = Image.open(img_path).convert('RGB')
        label = int(label)
        
        if self.transform:
            image = self.transform(image)

        return image, label

class RandomChannelDropout:
    def __init__(self):
        self.channel_combinations = [
            (0, 1, 2),  # RGB
            (0, 1),     # RG
            (0, 2),     # RB
            (1, 2),     # GB
            (0,),       # R
            (1,),       # G
            (2,)        # B
        ]

    def __call__(self, img):
        channels = list(img.split())
        chosen_combination = random.choice(self.channel_combinations)
        combined_img = Image.merge('RGB', [channels[i] if i in chosen_combination else Image.new('L', img.size) for i in range(3)])
        return combined_img

# Define transformations

#基礎影像處理
basic_augmentations = transforms.Compose([
    transforms.Resize(96),
    transforms.CenterCrop(96),
    transforms.ToTensor()
])

train_augmentations = transforms.Compose([
    transforms.RandomResizedCrop(96),
    transforms.RandomHorizontalFlip(1),
    transforms.RandomVerticalFlip(0.1),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.2),
    transforms.RandomRotation(15),
    RandomChannelDropout(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

val_augmentations = transforms.Compose([
    transforms.Resize(96),
    transforms.CenterCrop(96),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

test_augmentations = transforms.Compose([
    transforms.Resize(96),
    transforms.CenterCrop(96),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

class DynamicConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True):
        super(DynamicConv2D, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.bias = bias

        # 定義動態卷積核權重和偏置
        self.weight = nn.Parameter(torch.randn(out_channels, in_channels, kernel_size, kernel_size))
        if bias:
            self.bias = nn.Parameter(torch.randn(out_channels))
        else:
            self.register_parameter('bias', None)

    def forward(self, x):
        # 根據輸入的channel數量進行權重調整
        current_in_channels = x.size(1)
        if current_in_channels < self.in_channels:
            weight = self.weight[:, :current_in_channels, :, :]
        else:
            weight = self.weight
        
        # 應用卷積操作
        return F.conv2d(x, weight, self.bias, self.stride, self.padding)

is_train = True
model_path = "best_1.pt"
if is_train:
    # 選擇ResNet-18作為模型
    model = models.resnet18(pretrained=False)
    model.conv1 = DynamicConv2D(in_channels=3, out_channels=64, kernel_size=7, padding=1)
    model.fc = torch.nn.Linear(model.fc.in_features, 50)
    model.to(device)
else:
    model = models.resnet18(pretrained=False)
    model.conv1 = DynamicConv2D(in_channels=3, out_channels=64, kernel_size=7, padding=1)
    model.fc = torch.nn.Linear(model.fc.in_features, 50)
    model = torch.load(model_path)
    model.eval()


Exception in thread Thread-7:
Traceback (most recent call last):
  File "c:\Users\wen\anaconda3\envs\wen\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\wen\anaconda3\envs\wen\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\wen\anaconda3\envs\wen\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\wen\anaconda3\envs\wen\lib\subprocess.py", line 1495, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\wen\anaconda3\envs\wen\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa4 in position 6: invalid start byte
c:\Users\wen\anaconda3\envs\wen\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  

# Load Dataset

In [9]:
# Create dataset instances
train_dataset = ImageNetDataset(data_file='train.txt', root_dir='', transform=train_augmentations)
val_dataset = ImageNetDataset(data_file='val.txt', root_dir='', transform=val_augmentations)
test_dataset = ImageNetDataset(data_file='test.txt', root_dir='', transform=test_augmentations)

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 訓練DynamicConv2D ResNet18

In [4]:
epochs = 30
# batch_size = 32
# test_batch_size = 32
lr = 0.1
momentum = 0.9
weight_decay = 1e-4
net_name = 'dy_resnet'
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

def adjust_lr(optimizer, epoch):
    if epoch in [epochs*0.5, epochs*0.75, epochs*0.85]:
        for p in optimizer.param_groups:
            p['lr'] *= 0.1
            lr = p['lr']
        print('Change lr:'+str(lr))

def train(epoch):
    model.train()
    avg_loss = 0.
    train_acc = 0.
    adjust_lr(optimizer, epoch)
    train_loader_len = len(train_loader.dataset)
    train_loader_iter = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training Epoch #{}".format(epoch))
    
    for batch_idx, (data, target) in train_loader_iter:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()
        optimizer.step()
        train_loader_iter.set_postfix(loss=loss.item(), accuracy=100. * train_acc.item() / train_loader_len)
    
    print('Train Epoch: {}, Loss: {:.6f}, Accuracy: {:.2f}%'.format(epoch, avg_loss / len(train_loader), 100. * train_acc / train_loader_len))



def val(epoch):
    model.eval()
    test_loss = 0.
    correct = 0
    val_loader_iter = tqdm(val_loader, total=len(val_loader), desc="Validation Epoch #{}".format(epoch))
    
    with torch.no_grad():
        for data, label in val_loader_iter:
            data, label = data.to(device), label.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, label, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
            val_loader_iter.set_postfix(loss=test_loss / len(val_loader.dataset), accuracy=100. * correct.item() / len(val_loader.dataset))
    
    test_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print('Validation Set: Average Loss: {:.4f}, Accuracy: {:.2f}%'.format(test_loss, accuracy))

    return accuracy

if is_train:
    best_val_acc = 0.
    for i in range(epochs):
        train(i + 1)
        temp_acc = val(i + 1)
        if temp_acc > best_val_acc:
            best_val_acc = temp_acc
            torch.save(model.state_dict(), 'best_1.pt')
            print('Best Accuracy: {:.2f}%'.format(best_val_acc))

    print('Final Best Accuracy: {:.2f}%'.format(best_val_acc))

Training Epoch #1: 100%|██████████| 990/990 [03:36<00:00,  4.58it/s, accuracy=5.02, loss=3.75] 


Train Epoch: 1, Loss: 3.793766, Accuracy: 5.02%


Validation Epoch #1: 100%|██████████| 8/8 [00:00<00:00,  8.15it/s, accuracy=9.33, loss=3.6]  


Validation Set: Average Loss: 3.5951, Accuracy: 9.33%
Best Accuracy: 9.33%


Training Epoch #2: 100%|██████████| 990/990 [03:35<00:00,  4.60it/s, accuracy=8.63, loss=3.73] 


...

Validation Set: Average Loss: 1.1225, Accuracy: 65.11%


Training Epoch #30: 100%|██████████| 990/990 [02:34<00:00,  6.40it/s, accuracy=56.1, loss=1.98] 


Train Epoch: 30, Loss: 1.503509, Accuracy: 56.08%


Validation Epoch #30: 100%|██████████| 8/8 [00:00<00:00, 12.31it/s, accuracy=63.6, loss=1.13] 

Validation Set: Average Loss: 1.1327, Accuracy: 63.56%
Final Best Accuracy: 66.00%


# 載入DynamicConv2D ResNet18

In [17]:
model = models.resnet18(pretrained=False)
model.conv1 = DynamicConv2D(in_channels=3, out_channels=64, kernel_size=7, padding=1)
model.fc = torch.nn.Linear(model.fc.in_features, 50)
model.load_state_dict(torch.load("best_1.pt"))
model.to(device)
model.eval()

c:\Users\wen\anaconda3\envs\wen\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\wen\anaconda3\envs\wen\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): DynamicConv2D()
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1)

In [18]:
def select_channels(image, channels='RGB'):
    if channels == 'R':
        return image[0, :, :].unsqueeze(0)
    elif channels == 'G':
        return image[1, :, :].unsqueeze(0)
    elif channels == 'B':
        return image[2, :, :].unsqueeze(0)
    elif channels == 'RG':
        return image[0:2, :, :]
    elif channels == 'RB':
        return torch.stack([image[0, :, :], image[2, :, :]], dim=0)
    elif channels == 'GB':
        return image[1:, :, :]
    elif channels == 'RGB':
        return image
    else:
        raise ValueError('Invalid channel selection')

In [19]:
def rgb_dataloader(set_channel = 'RGB'):
    test_augmentations = transforms.Compose([
        transforms.Resize(96),
        transforms.CenterCrop(96),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.Lambda(lambda x: select_channels(x, set_channel))  # Change 'RGB' to other combinations as needed
    ])

    test_dataset = ImageNetDataset(data_file='test.txt', root_dir='', transform=test_augmentations)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    return test_loader

# 測試集驗證

In [21]:
def test(model, rgb_set):
    model.eval()
    correct = 0
    test_loader = rgb_dataloader(set_channel = rgb_set)
    test_loader_len = len(test_loader.dataset)
    test_loader_iter = tqdm(test_loader, total=len(test_loader), desc="Testing")
    
    all_preds = []
    all_targets = []
    
    start_time = time.time()
    
    with torch.no_grad():
        for data, target in test_loader_iter:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            
            test_loader_iter.set_postfix(accuracy=100. * correct.item() / test_loader_len)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    accuracy = 100. * correct / test_loader_len
    
    # 計算 Precision, Recall 和 F1-score
    precision = 100. * precision_score(all_targets, all_preds, average='macro')
    recall = 100. * recall_score(all_targets, all_preds, average='macro')
    f1 = 100. * f1_score(all_targets, all_preds, average='macro')
    
    # 計算 FLOPS
    flops = torchprofile.profile_macs(model, torch.randn(1, *data.shape[1:]).to(device))
    
    return accuracy.item(), precision, recall, f1, flops, elapsed_time

rgb_list = ["RGB", "RG", "RB", "GB", "R", "G", "B"]
for rgb in rgb_list:
    resnet_acc, resnet_precision, resnet_recall, resnet_f1, resnet_flops, resnet_elapsed_time = test(model,rgb)
    print(f"RGB Set: {rgb:s}, Accuracy: {resnet_acc:.2f}%, Precision: {resnet_precision:.2f}%, Recall: {resnet_recall:.2f}%, F1 Score: {resnet_f1:.2f}%, FLOPS: {resnet_flops:d}, Elapsed Time: {resnet_elapsed_time:.2f} seconds")
    # resnet18_acc, resnet18_elapsed_time = test(model,rgb)
    # print(f"RGB Set: {rgb:s}, Accuracy: {resnet18_acc:.2f}%, Elapsed Time: {resnet18_elapsed_time:.2f} seconds")

Testing: 100%|██████████| 8/8 [00:00<00:00,  9.87it/s, accuracy=69.6]


RGB Set: RGB, Accuracy: 69.56%, Precision: 71.25%, Recall: 69.56%, F1 Score: 68.36%, FLOPS: 1274698880, Elapsed Time: 0.81 seconds


Testing: 100%|██████████| 8/8 [00:00<00:00, 12.19it/s, accuracy=57.3]


RGB Set: RG, Accuracy: 57.33%, Precision: 61.72%, Recall: 57.33%, F1 Score: 56.81%, FLOPS: 1248155776, Elapsed Time: 0.66 seconds


Testing: 100%|██████████| 8/8 [00:00<00:00, 11.59it/s, accuracy=56.9]


...

Testing: 100%|██████████| 8/8 [00:00<00:00, 10.87it/s, accuracy=22.2]
c:\Users\wen\anaconda3\envs\wen\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


RGB Set: G, Accuracy: 22.22%, Precision: 31.10%, Recall: 22.22%, F1 Score: 20.10%, FLOPS: 1221612672, Elapsed Time: 0.74 seconds


Testing: 100%|██████████| 8/8 [00:00<00:00, 10.74it/s, accuracy=21.3]

RGB Set: B, Accuracy: 21.33%, Precision: 26.59%, Recall: 21.33%, F1 Score: 17.88%, FLOPS: 1221612672, Elapsed Time: 0.75 seconds



c:\Users\wen\anaconda3\envs\wen\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 載入ResNet18

In [7]:
model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 50)
model.to(device)
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# 訓練ResNet18

In [10]:
epochs = 30
# batch_size = 32
# test_batch_size = 32
lr = 0.1
momentum = 0.9
weight_decay = 1e-4
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

def adjust_lr(optimizer, epoch):
    if epoch in [epochs*0.5, epochs*0.75, epochs*0.85]:
        for p in optimizer.param_groups:
            p['lr'] *= 0.1
            lr = p['lr']
        print('Change lr:'+str(lr))

def train(epoch):
    model.train()
    avg_loss = 0.
    train_acc = 0.
    adjust_lr(optimizer, epoch)
    train_loader_len = len(train_loader.dataset)
    train_loader_iter = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training Epoch #{}".format(epoch))
    
    for batch_idx, (data, target) in train_loader_iter:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()
        optimizer.step()
        train_loader_iter.set_postfix(loss=loss.item(), accuracy=100. * train_acc.item() / train_loader_len)
    
    print('Train Epoch: {}, Loss: {:.6f}, Accuracy: {:.2f}%'.format(epoch, avg_loss / len(train_loader), 100. * train_acc / train_loader_len))



def val(epoch):
    model.eval()
    test_loss = 0.
    correct = 0
    val_loader_iter = tqdm(val_loader, total=len(val_loader), desc="Validation Epoch #{}".format(epoch))
    
    with torch.no_grad():
        for data, label in val_loader_iter:
            data, label = data.to(device), label.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, label, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
            val_loader_iter.set_postfix(loss=test_loss / len(val_loader.dataset), accuracy=100. * correct.item() / len(val_loader.dataset))
    
    test_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print('Validation Set: Average Loss: {:.4f}, Accuracy: {:.2f}%'.format(test_loss, accuracy))

    return accuracy

if is_train:
    best_val_acc = 0.
    for i in range(epochs):
        train(i + 1)
        temp_acc = val(i + 1)
        if temp_acc > best_val_acc:
            best_val_acc = temp_acc
            torch.save(model.state_dict(), 'resnet18_best.pt')
            print('Best Accuracy: {:.2f}%'.format(best_val_acc))

    print('Final Best Accuracy: {:.2f}%'.format(best_val_acc))

Training Epoch #1: 100%|██████████| 990/990 [02:17<00:00,  7.22it/s, accuracy=3.14, loss=3.75] 


Train Epoch: 1, Loss: 3.901848, Accuracy: 3.14%


Validation Epoch #1: 100%|██████████| 8/8 [00:00<00:00, 12.61it/s, accuracy=4.89, loss=3.72]


Validation Set: Average Loss: 3.7242, Accuracy: 4.89%
Best Accuracy: 4.89%


Training Epoch #2: 100%|██████████| 990/990 [01:57<00:00,  8.43it/s, accuracy=5.26, loss=3.62] 


...

Validation Set: Average Loss: 1.6510, Accuracy: 50.00%


Training Epoch #30: 100%|██████████| 990/990 [02:01<00:00,  8.17it/s, accuracy=42.2, loss=2.28]


Train Epoch: 30, Loss: 2.041643, Accuracy: 42.19%


Validation Epoch #30: 100%|██████████| 8/8 [00:00<00:00, 13.27it/s, accuracy=51.3, loss=1.58] 

Validation Set: Average Loss: 1.5810, Accuracy: 51.33%
Final Best Accuracy: 52.67%


# 載入訓練好ResNet18

In [11]:
resnet18_model = models.resnet18(pretrained=False)
resnet18_model.fc = torch.nn.Linear(model.fc.in_features, 50)
resnet18_model.load_state_dict(torch.load("resnet18_best.pt"))
resnet18_model.to(device)
resnet18_model.eval()

c:\Users\wen\anaconda3\envs\wen\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\wen\anaconda3\envs\wen\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
test_augmentations = transforms.Compose([
    transforms.Resize(96),
    transforms.CenterCrop(96),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

test_dataset = ImageNetDataset(data_file='test.txt', root_dir='', transform=test_augmentations)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 測試ResNet18

In [16]:
def test(model):
    model.eval()
    correct = 0
    test_loader_len = len(test_loader.dataset)
    test_loader_iter = tqdm(test_loader, total=len(test_loader), desc="Testing")
    
    all_preds = []
    all_targets = []
    
    start_time = time.time()
    
    with torch.no_grad():
        for data, target in test_loader_iter:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            
            test_loader_iter.set_postfix(accuracy=100. * correct.item() / test_loader_len)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    accuracy = 100. * correct / test_loader_len
    
    # 計算 Precision, Recall 和 F1-score
    precision = 100. * precision_score(all_targets, all_preds, average='macro')
    recall = 100. * recall_score(all_targets, all_preds, average='macro')
    f1 = 100. * f1_score(all_targets, all_preds, average='macro')
    
    # 計算 FLOPS
    flops = torchprofile.profile_macs(model, torch.randn(1, *data.shape[1:]).to(device))
    
    return accuracy.item(), precision, recall, f1, flops, elapsed_time

resnet_acc, resnet_precision, resnet_recall, resnet_f1, resnet_flops, resnet_elapsed_time = test(resnet18_model)
print(f"Accuracy: {resnet_acc:.2f}%, Precision: {resnet_precision:.2f}%, Recall: {resnet_recall:.2f}%, F1 Score: {resnet_f1:.2f}%, FLOPS: {resnet_flops:d}, Elapsed Time: {resnet_elapsed_time:.2f} seconds")

Testing: 100%|██████████| 8/8 [00:00<00:00, 12.99it/s, accuracy=51.3]


Accuracy: 51.33%, Precision: 52.68%, Recall: 51.33%, F1 Score: 50.48%, FLOPS: 333585408, Elapsed Time: 0.62 seconds
